<a href="https://colab.research.google.com/github/mk-imagine/csc869-g5-project/blob/main/CSC_869_Term_Project_Embeddings_and_Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dill
!pip install openai
!pip install tiktoken
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from google.colab import files
from sklearn.metrics.pairwise import cosine_similarity
import dill
import pandas as pd
import numpy as np
import os
#from langchain.llms import OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/CSC869/df_eval.pkl','rb') as file:
  df = dill.load(file)

print(df)

                                             target_raw  \
0     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
1     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
2     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
3     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
4     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
...                                                 ...   
8514  CS 51  (C-ID COMP 142)         3 units  \nIntr...   
8515  CIS020:\n\n3.0 Units\nPrint\nDescription:\n\nA...   
8516  CIS020:\n\n3.0 Units\nPrint\nDescription:\n\nA...   
8517  COMSC-2 3 UNITS\nComputer Assembly Language\nT...   
8518  Introduction to Assembly Language Programming\...   

                                             source_raw  \
0     MATH 3A - Calculus I\n5 Units, 5 lecture hours...   
1     MATH 109 - Calculus I\n4 Units, 4 Lecture hour...   
2     MAT 1140 - Calculus I\n4 Units\nPrerequisites:...   
3     MATH 141 - Calculus I\n4 Units\nGE Area B1\nPr...

In [ ]:
# Embedding with spacy
import spacy

nlp = spacy.load("en_core_web_sm")

target_raw_data = df['target_raw'].astype(str).tolist()
source_raw_data = df['source_raw'].astype(str).tolist()
target_topics_data = df['target_topics'].astype(str).tolist()
source_topics_data = df['source_topics'].astype(str).tolist()

def generate_embeddings(text_data):
    embeddings = []
    for text in text_data:
        doc = nlp(text)

        if doc.has_vector:
            embeddings.append(doc.vector)
        else:
            embeddings.append(np.zeros_like(doc.vector))
    return np.array(embeddings)

target_raw_embeddings = generate_embeddings(target_raw_data)
source_raw_embeddings = generate_embeddings(source_raw_data)
target_topics_embeddings = generate_embeddings(target_topics_data)
source_topics_embeddings = generate_embeddings(source_topics_data)

embedding_df = pd.DataFrame({
    'target_raw_embeddings': list(target_raw_embeddings),
    'source_raw_embeddings': list(source_raw_embeddings),
    'target_topics_embeddings': list(target_topics_embeddings),
    'source_topics_embeddings': list(source_topics_embeddings)
})

df_embeddings = pd.concat([df, embedding_df], axis =1)

print(df_embeddings)

                                             target_raw  \
0     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
1     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
2     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
3     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
4     MATH 226 - Calculus I\n4 Units\n\nGraphs. Diff...   
...                                                 ...   
8514  CS 51  (C-ID COMP 142)         3 units  \nIntr...   
8515  CIS020:\n\n3.0 Units\nPrint\nDescription:\n\nA...   
8516  CIS020:\n\n3.0 Units\nPrint\nDescription:\n\nA...   
8517  COMSC-2 3 UNITS\nComputer Assembly Language\nT...   
8518  Introduction to Assembly Language Programming\...   

                                             source_raw  \
0     MATH 3A - Calculus I\n5 Units, 5 lecture hours...   
1     MATH 109 - Calculus I\n4 Units, 4 Lecture hour...   
2     MAT 1140 - Calculus I\n4 Units\nPrerequisites:...   
3     MATH 141 - Calculus I\n4 Units\nGE Area B1\nPr...

In [ ]:
import numpy

print(numpy.shape(target_raw_embeddings))
print(numpy.shape(source_raw_embeddings))
print(numpy.shape(target_topics_embeddings))
print(numpy.shape(source_topics_embeddings))

embedding_dimensionality = target_raw_embeddings.shape[1]

print(f'Embedding Dimensionality: {embedding_dimensionality}')

(8519, 96)
(8519, 96)
(8519, 96)
(8519, 96)
Embedding Dimensionality: 96


In [ ]:
# Cosine Similarity

import numpy
from sklearn.metrics.pairwise import cosine_similarity


def compare_by_row():
    """ function that calls cosine_similarity on 1 sample of :column: dimensions. """
    cosine_sims = []
    for rows_ndx in range(num_rows):
       # if rows_ndx > 5: break

        source_data = source_raw_embeddings[rows_ndx, :].reshape(1, 96)
       # print(f'Source embedding: ({numpy.shape(source_data)})')
       # print(source_data)

        target_data = target_raw_embeddings[rows_ndx, :].reshape(1, 96)
      #  print(f'Target embedding: ({numpy.shape(target_data)})')
     #   print(target_data)

        cosine_sims.append(cosine_similarity(target_data, source_data)[0])

    return cosine_sims

def compare_matrices():
    """ function that calls cosine_similarity on :row: samples of :column: dimensions. """
    """ will give cartesian product """

  #  print(f'Source embedding: ({numpy.shape(source_raw_embeddings)})')
  #  print(source_raw_embeddings[:5, :20])

  #  print(f'Target embedding: ({numpy.shape(target_raw_embeddings)})')
  #  print(target_raw_embeddings[:5, :20])

    return cosine_similarity(target_raw_embeddings, source_raw_embeddings)


num_rows, num_columns = numpy.shape(target_raw_embeddings)

print('Data dimensions:')
print(f'\tRows: {num_rows}')
print(f'\tCols: {num_columns}')

cosine_sim = compare_by_row()

df_embeddings = pd.DataFrame({
     'target_raw_embeddings': list(target_raw_embeddings),
     'source_raw_embeddings': list(source_raw_embeddings),
     'raw_cosine_similarity': list(cosine_sim)
})
df_cosine_similarity = pd.concat([df, df_embeddings], axis =1)

print(cosine_sim)
print(numpy.shape(cosine_sim))

print(df_cosine_similarity)

Data dimensions:
	Rows: 8519
	Cols: 96
[array([0.81904936], dtype=float32), array([0.8093945], dtype=float32), array([0.7895809], dtype=float32), array([0.7921319], dtype=float32), array([0.8236138], dtype=float32), array([0.776265], dtype=float32), array([0.84320986], dtype=float32), array([0.73164594], dtype=float32), array([0.7970758], dtype=float32), array([0.7933459], dtype=float32), array([0.7597778], dtype=float32), array([0.8564751], dtype=float32), array([0.8209504], dtype=float32), array([0.92344856], dtype=float32), array([0.8004688], dtype=float32), array([0.7620703], dtype=float32), array([0.8765075], dtype=float32), array([0.7496196], dtype=float32), array([0.77233815], dtype=float32), array([0.7634496], dtype=float32), array([0.7493356], dtype=float32), array([0.77425593], dtype=float32), array([0.80509996], dtype=float32), array([0.8036306], dtype=float32), array([0.805508], dtype=float32), array([0.7034509], dtype=float32), array([0.7737476], dtype=float32), array([0.7

In [ ]:
# Cosine Similarity

import numpy
from sklearn.metrics.pairwise import cosine_similarity


def compare_by_row2():
    """ function that calls cosine_similarity on 1 sample of :column: dimensions. """
    cosine_sims2 = []
    for rows2_ndx in range(num_rows2):

        source_top_data = source_topics_embeddings[rows2_ndx, :].reshape(1, 96)

        target_top_data = target_topics_embeddings[rows2_ndx, :].reshape(1, 96)

        cosine_sims2.append(cosine_similarity(target_top_data, source_top_data)[0])

    return cosine_sims2

def compare_matrices2():
    """ function that calls cosine_similarity on :row: samples of :column: dimensions. """
    """ will give cartesian product """

    return cosine_similarity(target_topics_embeddings, source_topics_embeddings)


num_rows2, num_columns2 = numpy.shape(target_topics_embeddings)

print('Data dimensions:')
print(f'\tRows: {num_rows2}')
print(f'\tCols: {num_columns2}')

cosine_sim2 = compare_by_row2()

df_all_embeddings = pd.DataFrame({
     'target_raw_embeddings': list(target_raw_embeddings),
     'source_raw_embeddings': list(source_raw_embeddings),
     'target_topics_embeddings': list(target_topics_embeddings),
     'source_topics_embeddings': list(source_topics_embeddings),
     'raw_cosine_similarity': list(cosine_sim),
     'topics_cosine_similarity': list(cosine_sim2)
})

df_all_cosine_similarity = pd.concat([df, df_all_embeddings], axis =1)

print(cosine_sim2)
print(numpy.shape(cosine_sim2))

print(df_all_cosine_similarity)

Data dimensions:
	Rows: 8519
	Cols: 96
[array([0.9156343], dtype=float32), array([0.9360435], dtype=float32), array([0.959775], dtype=float32), array([0.99723554], dtype=float32), array([0.95464814], dtype=float32), array([0.9230439], dtype=float32), array([0.9869172], dtype=float32), array([0.9716481], dtype=float32), array([0.97653633], dtype=float32), array([0.9763756], dtype=float32), array([0.99138457], dtype=float32), array([0.94889903], dtype=float32), array([0.9339827], dtype=float32), array([0.9927131], dtype=float32), array([0.95775664], dtype=float32), array([0.9736892], dtype=float32), array([0.9258833], dtype=float32), array([0.9805563], dtype=float32), array([0.9679589], dtype=float32), array([0.99579924], dtype=float32), array([0.99204665], dtype=float32), array([0.9918086], dtype=float32), array([0.9391462], dtype=float32), array([0.98961973], dtype=float32), array([0.98478496], dtype=float32), array([0.95353204], dtype=float32), array([0.9145411], dtype=float32), array

In [ ]:
df_all_cosine_similarity.to_pickle('/content/drive/MyDrive/CSC869/df_all_embed_cosine_sim.pkl')

In [ ]:
# Skip
# Embeddings using OpenAIEmbeddings

# Would not work due to rate limit error for this specific model - free OpenAI API key maxes out at a rate limit of 3, which is not enough.
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

columns_to_embed = ['target_raw', 'source_raw']

openai.api_key =

# Initialize OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=
)

print(df.columns)

Index(['target_raw', 'source_raw', 'target_topics', 'source_topics', 'label'], dtype='object')


In [ ]:
import openai
openai .__version__

'1.3.8'

In [ ]:
# Skip

import openai
from openai import OpenAI
#import time

#time.sleep(60)

for column in columns_to_embed:
    # Combine the tokenized tokens into sentences
    df[column + '_text'] = df[column].apply(lambda tokens: ' '.join(tokens))

    # Embed each text separately
    df[column + '_embeddings'] = [embeddings.embed_query(text) for text in df[column + '_text'].tolist()]



RateLimitError: ignored

In [ ]:
# Skip
# Might work for openai embedding
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

cosine_similarities = []

for i in range(len(df)):
  target_raw_embeddings = df['target_raw_embeddings'][i]
  source_raw_embeddings = df['source_raw_embeddings'][i]

  similarity_score = cosine_similarity([target_raw_embeddings],[source_raw_embeddings])[0,0]
  cosine_similarities.append(similarity_score)

df['cosine_similarity'] = cosine_similarities



KeyError: ignored